In [2]:
import numpy as np
import pandas as pd
import keras
import cv2

C:\Users\Sundeep\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train = x_train.reshape((60000, 28 * 28))
x_test = x_test.reshape((10000, 28 * 28))

In [5]:
x_test[1].shape

(784,)

In [6]:
x_train = x_train / 255.0

x_test = x_test / 255.0

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten
from keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split

In [8]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=(28*28)))
model.add(keras.layers.Dropout(0.3))
model.add(Dense(64, activation='relu', name = 'features'))
# model.add(keras.layers.Dropout(0.3))
model.add(Dense(32, activation='relu', name = 'features1'))
model.add(Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
features (Dense)             (None, 64)                8256      
_________________________________________________________________
features1 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 111,146
Trainable params: 111,146
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train, epochs=15)

Epoch 1/15
60000/60000 [==============================] - 10s 167us/step - loss: 0.3266 - acc: 0.9013
Epoch 2/15
60000/60000 [==============================] - 9s 155us/step - loss: 0.1625 - acc: 0.9503
Epoch 3/15
60000/60000 [==============================] - 9s 155us/step - loss: 0.1316 - acc: 0.9588
Epoch 4/15
60000/60000 [==============================] - 9s 154us/step - loss: 0.1161 - acc: 0.9643
Epoch 5/15
60000/60000 [==============================] - 9s 154us/step - loss: 0.1026 - acc: 0.9682
Epoch 6/15
60000/60000 [==============================] - 9s 153us/step - loss: 0.0939 - acc: 0.9708
Epoch 7/15
60000/60000 [==============================] - 9s 154us/step - loss: 0.0897 - acc: 0.9712
Epoch 8/15
60000/60000 [==============================] - 10s 159us/step - loss: 0.0833 - acc: 0.9741
Epoch 9/15
60000/60000 [==============================] - 9s 156us/step - loss: 0.0769 - acc: 0.9749
Epoch 10/15
60000/60000 [==============================] - 9s 155us/step - loss: 0.0732 -

In [11]:
np.argmax(model.predict(x_test[1].reshape(1,784)))

2

In [13]:
def get_img_contour_thresh(img):
    x, y, w, h = 100, 75, 300, 300
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (35, 35), 0)
    _, thresh = cv2.threshold(blur, 10, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh = thresh[y:y + h, x:x + w]
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    return img, contours, thresh

In [15]:
cap = cv2.VideoCapture(0)

while (cap.isOpened()):
    ret, img = cap.read()
    img, contours, thresh = get_img_contour_thresh(img)

    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(contour) > 10:

            x, y, w, h = cv2.boundingRect(contour)

            newImage = thresh[y:y + h, x:x + w]
            newImage = cv2.resize(newImage, (28, 28))
            newImage = np.array(newImage)
            newImage = newImage.flatten()

            pred_ = np.argmax(model.predict(newImage.reshape(1,784)))

    x, y, w, h = 100, 75, 300, 300
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(img, "Detected Number is: " + str(pred_), (10, 320),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Frame", img)
    cv2.imshow("Contours", thresh)
    k = cv2.waitKey(10)
    if k == 27:
        break